In [1]:
import pandas as pd
from io import StringIO 
from geopandas import GeoDataFrame
from iclientpy.rest.api.model import DataItemType,PermissionType,DataPermissionType
from iclientpy.portal import Portal,BaseLayerType,MapShareSettingBuilder,DataShareSettingBuilder

In [2]:
portal=Portal('http://localhost:8090/iportal','admin','Supermap123')

从CSV文件读取数据

In [3]:
df=pd.read_csv("province.csv")
df["2015年"]=pd.to_numeric(df["2015年"],errors='coerce')

获取地理位置数据，geonjson格式

In [4]:
gdf=GeoDataFrame.from_file('./data.json')

将CSV数据与地理位置数据进行拼接，根据省份名称所在字段名进行数据拼接

In [5]:
result=gdf.merge(df,left_on='name',right_on='地区')
result.drop(['name'],axis=1,inplace=True)

上传数据到iportal

In [6]:
data_id = portal.upload_dataframe_as_json('all.json',result)

配置数据的共享信息

In [7]:
data_share=DataShareSettingBuilder().share_to_everyone(DataPermissionType.DOWNLOAD).build()

In [8]:
portal.config_data_sharesetting(data_id,data_share)

将数据转换为Layer，便于生成地图

In [9]:
layer=portal.prepare_geojson_layer(data_id,'layer')

在iportal上根据Layer创建地图，默认底图为天地图

In [10]:
map_id=portal.create_map(layers=[layer],epsgCode=3857,map_title='overlay')

搜索名称为share的群组，配置地图的共享权限信息

In [11]:
groups=portal.search_group(keywords=['share'])

In [12]:
map_share=MapShareSettingBuilder().share_to_group(groups[0].id,PermissionType.READWRITE).build()

In [13]:
portal.config_map_sharesetting(map_id,map_share)